In [1]:
import os
import ffmpeg

In [2]:
def merge_videos_and_create_timestamps(folder_path):
    """
    Merges all MP4 videos in a folder into a single video and creates a timestamp file using ffmpeg-python.

    Args:
        folder_path (str): The path to the folder containing the MP4 videos.
    """

    video_files = []
    timestamps = []
    total_duration = 0

    # Get all MP4 files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4"):
            filepath = os.path.join(folder_path, filename)
            video_files.append(filepath)

            # Get duration of the video using ffprobe (part of ffmpeg)
            probe = ffmpeg.probe(filepath)
            duration = float(probe['format']['duration'])
            timestamps.append((filename, total_duration))
            total_duration += duration

    # Create the ffmpeg command to concatenate the videos with explicit audio mapping
    inputs = [ffmpeg.input(file) for file in video_files]
    concat_filter = f"concat=n={len(video_files)}:v=1:a=1"
    merged_video = ffmpeg.concat(*inputs, v=1, a=1) \
        .output(os.path.join(folder_path, "merged_video.mp4")) \
        .global_args('-map', '0:v:0', '-map', '0:a:0')  # Map video and audio from the first input

    # Run the ffmpeg command and capture stderr output
    try:
        merged_video.run(capture_stdout=True, capture_stderr=True)
    except ffmpeg.Error as e:
        print(f"FFmpeg error occurred: {e.stderr.decode()}")
        return  # Stop further execution if there's an error

    # Write the timestamps to a text file
    timestamp_file_path = os.path.join(folder_path, "timestamps.txt")
    with open(timestamp_file_path, "w") as f:
        for filename, timestamp in timestamps:
            f.write(f"{filename} - {int(timestamp // 60):02d}:{int(timestamp % 60):02d}\n")

    print(f"Merged video saved to: {os.path.join(folder_path, 'merged_video.mp4')}")
    print(f"Timestamps saved to: {timestamp_file_path}")

In [3]:
# Get the folder path 
folder_path = r"D:\merge-test\Journey"

# Merge videos and create timestamps
merge_videos_and_create_timestamps(folder_path)

Merged video saved to: D:\merge-test\Journey\merged_video.mp4
Timestamps saved to: D:\merge-test\Journey\timestamps.txt
